Library Imports

In [2]:
from __future__ import print_function

import numpy as np
import pandas as pd
import recordlinkage as rl
import pickle
from recordlinkage.index import Block
from recordlinkage.index import Full
from recordlinkage.index import SortedNeighbourhood
from recordlinkage.compare import Exact, String
from recordlinkage.datasets import load_febrl4

Add LW column names as variables since they have changed in the past

In [7]:
#Link File Field Variables
LF_CN = 'Organisation Name' #Company Name Field
LF_CNU = 'Company/Charity Number'
LF_A1 = 'Address Line 1'
LF_A2 = 'Address Line 2'
LF_CITY = 'City'
LF_COUNTY = 'State/Province'
LF_PC = 'Postal Code'

Data Import
df_LW in seperate block for quicker re-import
date added as column 'Processing Date'

In [8]:
df_LW = pd.read_csv(r"C:\Users\Clamfighter\Machine_Learning_Project\my_env\Heggle\Record Linkage - Company Data\Company Data\RLW_SEPT.csv",encoding = "ISO-8859-1" ) 
df_BCD = pd.read_csv(r"C:\Users\Clamfighter\Machine_Learning_Project\my_env\Heggle\Record Linkage - Company Data\Company Data\BCD.csv",encoding = "ISO-8859-1",low_memory=False) 


In [9]:
df_LW = pd.read_csv(r"C:\Users\Clamfighter\Machine_Learning_Project\my_env\Heggle\Record Linkage - Company Data\Company Data\RLW_SEPT.csv",encoding = "ISO-8859-1" ) 
df_LW['Processing Date'] = pd.to_datetime('today').strftime("%m/%d/%Y")

df_LW

,Organisation ID,Company/Charity Number,Organisation Name,Website,Region,Sector,Industry,Visible on the Employer Map,Employer Summary,Consumer Tags,Address Line 1,Address Line 2,City,State/Province,Postal Code,Opted into B2B directory?,B2B Directory Category,B2B summary,B2B phone number,Processing Date
0,0012I00002fejxF,122689,MACAULAY ASKERNISH LTD,www.askernishquarry.com,Scotland,Private Sector,Construction,1,"We are a Civil Engineering, Construction & Uti...",NaN,Lochboisdale,NaN,Isle of South Uist,NaN,HS8 5TH,1,Construction,We offer construction services,1878700278,09/30/2021
1,0010G000024CJYf,207773,the Charterhouse,https://thecharterhouse.org/,London,Third Sector,Charity,1,"Our mission is to provide housing, support and...",NaN,Charterhouse Square,NaN,London,NaN,EC1M 6AN,1,Events/venues,The Charterhouse is the ideal historical venue...,0207 253 9503,09/30/2021
2,0012I00002YTthr,223041,Air Liquide Healthcare Limited,https://www.airliquidehealthcare.co.uk/,West Midlands,Private Sector,Health,1,Air Liquide Healthcare: Is a leading healthcar...,#healthcare #healthprovider #oxygen #diabetes ...,Alpha House,Wassage Way,Droitwich,NaN,WR9 0NX,0,NaN,NaN,NaN,09/30/2021
3,0010G000029pEzN,234216,Birmingham Diocesan Trust,www.birminghamdiocese.org.uk,West Midlands,Third Sector,Charity,1,Faith based charity.,NaN,Cathedral House,Queensway,BIRMINGHAM,UK,B4 6EX,0,NaN,NaN,NaN,09/30/2021
4,0012I00002gRxSB,279699,North Kensington Law Centre,https://nklc.org.uk,London,Third Sector,Charity,1,Law Centre providing specialist advice in the ...,#Law Centres,NORTH KENSINGTON LAW CENTRE,NaN,LONDON,NaN,W11 4AT,0,NaN,NaN,NaN,09/30/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,0012I00002QIhsd,NaN,Society Matters cic,www.societymatterscic.com,North East England,Third Sector,Social Enterprise,1,Society Matters delivers services for society ...,Training\nWelfare Benefits Training\nUniversal...,The Davidson Building,Swan Street,Gateshead,Tyne & Wear,NE8 1BG,1,Recruitment & training,"Social enterprise, trading arm of Citizens Adv...",0191 478 5100 X 280,09/30/2021
211,0010G00002JMXIm,NaN,1Point (North West) Ltd,www.1pointbolton.org.uk,North West England,Third Sector,Health,1,1point's vision is to create positive partners...,#counselling #therapy #IAPT #wellbeing #talkin...,Silverwell House,1 Silverwell Lane,Bolton,NaN,BL1 1QN,1,Health & care,Free assessment for Counselling to patients re...,1204917744,09/30/2021
212,0012I00002R97ze,NaN,Smiths (Harlow) Ltd,www.smiths-harlow.co.uk,South East England,Private Sector,Engineering,1,Precision Engineering & Complex Component Mach...,#Engineering #Aerospace,Barrows Road,NaN,Harlow,NaN,CM19 5AT,1,Health & care,Precision Engineering & Complex Component Mach...,01279 425641,09/30/2021
213,0012I00002TW5J4,NaN,HP Landscapes,https://www.hplandscapes.com/,London,Private Sector,Professional Services,1,Garden maintenance and landscaping company,NaN,4 Broadway Court,NaN,London,NaN,SW19 1RG,1,Building services & facilities management,Garden maintenance and landscaping company bas...,020 7731 3552,09/30/2021


Drop unwanted columns from BCD dataframe

In [10]:
df_BCD = df_BCD.drop(df_BCD.columns[10:], axis=1)
df_BCD = df_BCD.drop(['RegAddress#CareOf',
                      'RegAddress#POBox',
                     'RegAddress#Country'], axis=1)
df_BCD

,CompanyName,CompanyNumber,RegAddress#AddressLine1,RegAddress#AddressLine2,RegAddress#PostTown,RegAddress#County,RegAddress#PostCode
0,! LTD,8209948,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,LS10 2RU
1,!? LTD,11399177,THE STUDIO HATHERLOW HOUSE,HATHERLOW,ROMILEY,NaN,SK6 3DY
2,!BIG IMPACT GRAPHICS LIMITED,11743365,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,EC1V 9LT
3,!L PRODUCTIONS LIMITED,12402527,95 MARMION AVENUE,NaN,CHINGFORD,NaN,E4 8EJ
4,!NKED LTD,12234705,29 CORRY DRIVE,NaN,LONDON,NaN,SW9 8QS
...,...,...,...,...,...,...,...
4506996,JASJ LIMITED,SC459332,5 CARDEN PLACE,NaN,ABERDEEN,NaN,AB10 1UT
4506997,JASJ PARTNERSHIP LLP,OC427693,20 LEADALE AVENUE,NaN,LONDON,NaN,E4 8AT
4506998,JASJACK CYCLES LTD,7763816,5 CROSS CLOSE,FREMINGTON,BARNSTAPLE,NaN,EX31 3AN
4506999,JASJEEV LIMITED,6927959,18 ALUM CLOSE,NaN,COVENTRY,WEST MIDLANDS,CV6 5TQ


Normalise the LW names, but keep the originals

In [11]:
normalised_names = []
to_remove = ['\([^()]*\)','ltd','Ltd','LTD', 'Limited', 'limited','LIMITED',' Group','GROUP','Holdings','HOLDINGS','LLP']
for i in df_LW[LF_CN]:
    for j in to_remove:
        i = i.replace(j, '')
    i.rstrip()
    normalised_names.append(i)
df_LW['Normalised Organisation Name'] = normalised_names
#del df_LW[LF_CN]
df_LW

,Organisation ID,Company/Charity Number,Organisation Name,Website,Region,Sector,Industry,Visible on the Employer Map,Employer Summary,Consumer Tags,...,Address Line 2,City,State/Province,Postal Code,Opted into B2B directory?,B2B Directory Category,B2B summary,B2B phone number,Processing Date,Normalised Organisation Name
0,0012I00002fejxF,122689,MACAULAY ASKERNISH LTD,www.askernishquarry.com,Scotland,Private Sector,Construction,1,"We are a Civil Engineering, Construction & Uti...",NaN,...,NaN,Isle of South Uist,NaN,HS8 5TH,1,Construction,We offer construction services,1878700278,09/30/2021,MACAULAY ASKERNISH
1,0010G000024CJYf,207773,the Charterhouse,https://thecharterhouse.org/,London,Third Sector,Charity,1,"Our mission is to provide housing, support and...",NaN,...,NaN,London,NaN,EC1M 6AN,1,Events/venues,The Charterhouse is the ideal historical venue...,0207 253 9503,09/30/2021,the Charterhouse
2,0012I00002YTthr,223041,Air Liquide Healthcare Limited,https://www.airliquidehealthcare.co.uk/,West Midlands,Private Sector,Health,1,Air Liquide Healthcare: Is a leading healthcar...,#healthcare #healthprovider #oxygen #diabetes ...,...,Wassage Way,Droitwich,NaN,WR9 0NX,0,NaN,NaN,NaN,09/30/2021,Air Liquide Healthcare
3,0010G000029pEzN,234216,Birmingham Diocesan Trust,www.birminghamdiocese.org.uk,West Midlands,Third Sector,Charity,1,Faith based charity.,NaN,...,Queensway,BIRMINGHAM,UK,B4 6EX,0,NaN,NaN,NaN,09/30/2021,Birmingham Diocesan Trust
4,0012I00002gRxSB,279699,North Kensington Law Centre,https://nklc.org.uk,London,Third Sector,Charity,1,Law Centre providing specialist advice in the ...,#Law Centres,...,NaN,LONDON,NaN,W11 4AT,0,NaN,NaN,NaN,09/30/2021,North Kensington Law Centre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,0012I00002QIhsd,NaN,Society Matters cic,www.societymatterscic.com,North East England,Third Sector,Social Enterprise,1,Society Matters delivers services for society ...,Training\nWelfare Benefits Training\nUniversal...,...,Swan Street,Gateshead,Tyne & Wear,NE8 1BG,1,Recruitment & training,"Social enterprise, trading arm of Citizens Adv...",0191 478 5100 X 280,09/30/2021,Society Matters cic
211,0010G00002JMXIm,NaN,1Point (North West) Ltd,www.1pointbolton.org.uk,North West England,Third Sector,Health,1,1point's vision is to create positive partners...,#counselling #therapy #IAPT #wellbeing #talkin...,...,1 Silverwell Lane,Bolton,NaN,BL1 1QN,1,Health & care,Free assessment for Counselling to patients re...,1204917744,09/30/2021,1Point (North West)
212,0012I00002R97ze,NaN,Smiths (Harlow) Ltd,www.smiths-harlow.co.uk,South East England,Private Sector,Engineering,1,Precision Engineering & Complex Component Mach...,#Engineering #Aerospace,...,NaN,Harlow,NaN,CM19 5AT,1,Health & care,Precision Engineering & Complex Component Mach...,01279 425641,09/30/2021,Smiths (Harlow)
213,0012I00002TW5J4,NaN,HP Landscapes,https://www.hplandscapes.com/,London,Private Sector,Professional Services,1,Garden maintenance and landscaping company,NaN,...,NaN,London,NaN,SW19 1RG,1,Building services & facilities management,Garden maintenance and landscaping company bas...,020 7731 3552,09/30/2021,HP Landscapes


Normalise BCD names, keeping the original names

In [12]:
normalised_names = []
to_remove = ['\([^()]*\)','ltd','Ltd','LTD', 'Limited', 'limited','LIMITED',' Group','GROUP','Holdings','HOLDINGS','LLP']
for i in df_BCD['CompanyName']:
    for j in to_remove:
        i = i.replace(j, '')
    i.rstrip()
    normalised_names.append(i)
df_BCD['Normalised Names'] = normalised_names
#del df_BCD['CompanyName']
df_BCD

,CompanyName,CompanyNumber,RegAddress#AddressLine1,RegAddress#AddressLine2,RegAddress#PostTown,RegAddress#County,RegAddress#PostCode,Normalised Names
0,! LTD,8209948,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,LS10 2RU,!
1,!? LTD,11399177,THE STUDIO HATHERLOW HOUSE,HATHERLOW,ROMILEY,NaN,SK6 3DY,!?
2,!BIG IMPACT GRAPHICS LIMITED,11743365,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,EC1V 9LT,!BIG IMPACT GRAPHICS
3,!L PRODUCTIONS LIMITED,12402527,95 MARMION AVENUE,NaN,CHINGFORD,NaN,E4 8EJ,!L PRODUCTIONS
4,!NKED LTD,12234705,29 CORRY DRIVE,NaN,LONDON,NaN,SW9 8QS,!NKED
...,...,...,...,...,...,...,...,...
4506996,JASJ LIMITED,SC459332,5 CARDEN PLACE,NaN,ABERDEEN,NaN,AB10 1UT,JASJ
4506997,JASJ PARTNERSHIP LLP,OC427693,20 LEADALE AVENUE,NaN,LONDON,NaN,E4 8AT,JASJ PARTNERSHIP
4506998,JASJACK CYCLES LTD,7763816,5 CROSS CLOSE,FREMINGTON,BARNSTAPLE,NaN,EX31 3AN,JASJACK CYCLES
4506999,JASJEEV LIMITED,6927959,18 ALUM CLOSE,NaN,COVENTRY,WEST MIDLANDS,CV6 5TQ,JASJEEV


Make the normlised names all Upper Case since comparison is case sensitive

In [13]:
#print(df_LW)
#print(df_LW[LF_A2].str.upper())
df_LW['Normalised Organisation Name'] = df_LW['Normalised Organisation Name'].str.upper()
df_LW[LF_A1] = df_LW[LF_A1].str.upper()
df_LW[LF_A2] = df_LW[LF_A2].str.upper()
df_LW[LF_CITY] = df_LW[LF_CITY].str.upper()
df_LW[LF_COUNTY] = df_LW[LF_COUNTY].str.upper()
df_BCD['Normalised Names'] = df_BCD['Normalised Names'].str.upper()

Reset the index of both dataframes

In [47]:
df_LW = df_LW.reset_index()
df_BCD = df_BCD.reset_index()

Finding true links between the datasets that don't need ML

In [16]:
true_indexer = rl.Index()

true_indexer.add(SortedNeighbourhood(left_on=LF_CN, right_on='CompanyName',window=1))

true_links = true_indexer.index(df_LW, df_BCD)

true_links

MultiIndex([(20, 2668193)],
           )

Finding the candidate links that may be matches
Generates these links by the similarit of the normalised names

In [19]:
indexer = rl.Index()

indexer.add(SortedNeighbourhood(left_on='Normalised Organisation Name', right_on='Normalised Names',window=3))

candidate_links = indexer.index(df_LW, df_BCD)

candidate_links

MultiIndex([(  0, 2113374),
            (  1, 3583479),
            (  2,  191155),
            (  3,  550543),
            (  4, 2440645),
            (  5, 2636450),
            (  6, 4031523),
            (  7,  370494),
            (  8, 3709858),
            (  9, 3346753),
            ...
            (205, 1660952),
            (206, 3826796),
            (207, 1843156),
            (208, 2770394),
            (209, 2432341),
            (210, 3308236),
            (211,   18263),
            (212, 3297527),
            (213, 4320257),
            (214, 1563778)],
           length=515)

Delete's true links from the candidate links

In [45]:
candidate_links = [x for x in candidate_links if x not in true_links]

For all candidate links, generate comparison values for specific columns. These are the features that will be ran through the model

In [20]:
comparer = rl.Compare()
comparer.add(String('Normalised Organisation Name', 'Normalised Names', method='jarowinkler',label='SIM Organisation Name'))#pass
comparer.add(String(LF_CNU, 'CompanyNumber', method='jarowinkler',label='SIM CompanyNumber'))
comparer.add(String(LF_A1, 'RegAddress#AddressLine1', method='jarowinkler',label='SIM Address Line 1'))
comparer.add(String(LF_A2, 'RegAddress#AddressLine2', method='jarowinkler',label='SIM Address Line 2'))
comparer.add(Exact(LF_CITY, 'RegAddress#PostTown', label='City'))
comparer.add(Exact(LF_COUNTY, 'RegAddress#County', label='County'))
comparer.add(Exact('Postal Code', 'RegAddress#PostCode', label='Post Code'))#fail


features = comparer.compute(candidate_links, df_LW, df_BCD)
print('feature shape', features.shape)

feature shape (515, 7)


In [21]:
features

,,SIM Organisation Name,SIM CompanyNumber,SIM Address Line 1,SIM Address Line 2,City,County,Post Code
0,2113374,0.990000,0.916667,0.588095,0.000000,0,0,1
1,3583479,0.903226,0.000000,0.500975,0.000000,0,0,0
2,191155,0.865125,0.436508,0.396465,0.296296,0,0,0
3,550543,0.895135,0.539683,0.488675,0.494517,1,0,0
4,2440645,0.925599,0.436508,0.563610,0.000000,0,0,0
...,...,...,...,...,...,...,...,...
210,3308236,0.858373,0.000000,0.000000,0.000000,0,0,0
211,18263,0.600000,0.000000,0.515432,0.402778,0,0,0
212,3297527,0.877115,0.000000,0.587302,0.000000,0,0,0
213,4320257,0.612027,0.000000,0.545833,0.000000,0,0,0


Import the pretrained model, and run the feature set through it, generating a list of labels

In [22]:
from sklearn import svm


with open(r'Models\RL_M7_SVM(RLW).pkl', 'rb') as file:
    model = pickle.load(file)
output = model.predict(features)


For all candidate links, combine the feature set, RLW data and BCD data together into one dataset (Thic could be fore efficient)
Add the output of the model as a columns

In [24]:
features_columns = ['SIM Organisation Name','SIM CompanyNumber','SIM Address Line 1','SIM Address Line 2','Exact City','Exact County','Exact Post Code']
LW_columns = (df_LW.columns.values)
BCD_columns = (df_BCD.columns.values)
combined_columns = np.concatenate([features_columns,LW_columns,BCD_columns])

df_combined = []
j=1
df_combined = pd.DataFrame(columns = combined_columns)


for i, row in features.iterrows():
    LW_list = df_LW.iloc[(i[0])].tolist()
    BCD_list = df_BCD.iloc[(i[1])].tolist()
    Combined_list = row.tolist() + LW_list + BCD_list 
    df_combined.loc[len(df_combined)] = Combined_list
    print(j)
    '''if j ==5:
        break'''
    j+=1
df_combined["label"] = output
df_combined

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,SIM Organisation Name,SIM CompanyNumber,SIM Address Line 1,SIM Address Line 2,Exact City,Exact County,Exact Post Code,index,Organisation ID,Company/Charity Number,...,Normalised Organisation Name,index,CompanyName,CompanyNumber,RegAddress#AddressLine1,RegAddress#AddressLine2,RegAddress#PostTown,RegAddress#County,RegAddress#PostCode,Normalised Names
0,0.990000,0.916667,0.588095,0.000000,0.0,0.0,1.0,0,0012I00002fejxF,122689,...,MACAULAY ASKERNISH,2113374,MACAULAY ASKERNISH LTD.,SC122689,HILLSIDE GARAGE,LOCHBOISDALE,SOUTH UIST,NaN,HS8 5TH,MACAULAY ASKERNISH .
1,0.903226,0.000000,0.500975,0.000000,0.0,0.0,0.0,1,0010G000024CJYf,207773,...,THE CHARTERHOUSE,3583479,THE CHARTERHOUSE ACCOMMODATION LTD,11148049,85 ST. MARTINS ROAD,NaN,COVENTRY,NaN,CV3 6ES,THE CHARTERHOUSE ACCOMMODATION
2,0.865125,0.436508,0.396465,0.296296,0.0,0.0,0.0,2,0012I00002YTthr,223041,...,AIR LIQUIDE HEALTHCARE,191155,AIR LIQUIDE OIL AND GAS SERVICES LIMITED,8811231,STATION ROAD,COLESHILL,BIRMINGHAM,WEST MIDLANDS,B46 1JY,AIR LIQUIDE OIL AND GAS SERVICES
3,0.895135,0.539683,0.488675,0.494517,1.0,0.0,0.0,3,0010G000029pEzN,234216,...,BIRMINGHAM DIOCESAN TRUST,550543,BIRMINGHAM DISABILITY RESOURCE CENTRE,2897250,"UNIT 18, ACE BUSINESS PARK","ACE BUSINESS PARK, MACKADOWN LANE",BIRMINGHAM,NaN,B33 0LD,BIRMINGHAM DISABILITY RESOURCE CENTRE
4,0.925599,0.436508,0.563610,0.000000,0.0,0.0,0.0,4,0012I00002gRxSB,279699,...,NORTH KENSINGTON LAW CENTRE,2440645,NORTH KENSINGTON RESIDENTIAL LTD,6066646,45 WARWICK STREET,NaN,WORTHING,WEST SUSSEX,BN11 3DQ,NORTH KENSINGTON RESIDENTIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,0.858373,0.000000,0.000000,0.000000,0.0,0.0,0.0,210,0012I00002QIhsd,NaN,...,SOCIETY MATTERS CIC,3308236,SOCIETY INC,CE016127,NaN,NaN,NaN,NaN,NaN,SOCIETY INC
511,0.600000,0.000000,0.515432,0.402778,0.0,0.0,0.0,211,0010G00002JMXIm,NaN,...,1POINT (NORTH WEST),18263,1PMOBILE LIMITED,10380951,NETWORK HQ 508 EDGWARE ROAD,THE HYDE,LONDON,ENGLAND,NW9 5AB,1PMOBILE
512,0.877115,0.000000,0.587302,0.000000,0.0,0.0,0.0,212,0012I00002R97ze,NaN,...,SMITHS (HARLOW),3297527,SMITHS (GLOUCESTER) LIMITED,4004654,ALKERTON COURT,EASTINGTON,STONEHOUSE,GLOUCESTERSHIRE,GL10 3AQ,SMITHS (GLOUCESTER)
513,0.612027,0.000000,0.545833,0.000000,0.0,0.0,0.0,213,0012I00002TW5J4,NaN,...,HP LANDSCAPES,4320257,HP KITCHENS LIMITED,11155068,"1ST FLOOR, CASHS BUSINESS CENTRE",228 WIDDRINGTON ROAD,COVENTRY,WEST MIDLANDS,CV1 4PB,HP KITCHENS


Do the same for true links, label them 1 automatically

In [40]:
for i, row in true_links:
    print(i)
    print(j)
    LW_list = df_LW.iloc[i].tolist()
    BCD_list = df_BCD.iloc[row].tolist()
    print(LW_list)
    print(BCD_list)
    Combined_list = ["","","","","","",""] + LW_list + BCD_list + ['1']
    df_combined.loc[len(df_combined)] = Combined_list
    print(j)
    '''if j ==5:
        break'''
    j+=1
df_combined

20
519
[20, '0012I00002gSF0r', '1432936', 'PB DESIGN AND DEVELOPMENTS LIMITED', 'https://www.pbdesign.co.uk/', 'South West England', 'Private Sector', 'Manufacturing', 1, 'design through production, testing, installation, commissioning and maintenance of electrical chargers & equipment.', nan, 'HITHER GREEN INDUSTRIAL ESTATE', nan, 'CLEVEDON', nan, 'BS21 6XT', 0, nan, nan, nan, '09/30/2021', 'PB DESIGN AND DEVELOPMENTS ']
[2668193, 'PB DESIGN AND DEVELOPMENTS LIMITED', '1432936', 'UNITS 9-10', 'HITHER GREEN INDUSTRIAL ESTATE', 'CLEVEDON', 'BRISTOL', 'BS21 6XT', 'PB DESIGN AND DEVELOPMENTS ']
519


,SIM Organisation Name,SIM CompanyNumber,SIM Address Line 1,SIM Address Line 2,Exact City,Exact County,Exact Post Code,index,Organisation ID,Company/Charity Number,...,index,CompanyName,CompanyNumber,RegAddress#AddressLine1,RegAddress#AddressLine2,RegAddress#PostTown,RegAddress#County,RegAddress#PostCode,Normalised Names,label
0,0.99,0.916667,0.588095,0,0,0,1,0,0012I00002fejxF,122689,...,2113374,MACAULAY ASKERNISH LTD.,SC122689,HILLSIDE GARAGE,LOCHBOISDALE,SOUTH UIST,NaN,HS8 5TH,MACAULAY ASKERNISH .,1
1,0.903226,0,0.500975,0,0,0,0,1,0010G000024CJYf,207773,...,3583479,THE CHARTERHOUSE ACCOMMODATION LTD,11148049,85 ST. MARTINS ROAD,NaN,COVENTRY,NaN,CV3 6ES,THE CHARTERHOUSE ACCOMMODATION,0
2,0.865125,0.436508,0.396465,0.296296,0,0,0,2,0012I00002YTthr,223041,...,191155,AIR LIQUIDE OIL AND GAS SERVICES LIMITED,8811231,STATION ROAD,COLESHILL,BIRMINGHAM,WEST MIDLANDS,B46 1JY,AIR LIQUIDE OIL AND GAS SERVICES,0
3,0.895135,0.539683,0.488675,0.494517,1,0,0,3,0010G000029pEzN,234216,...,550543,BIRMINGHAM DISABILITY RESOURCE CENTRE,2897250,"UNIT 18, ACE BUSINESS PARK","ACE BUSINESS PARK, MACKADOWN LANE",BIRMINGHAM,NaN,B33 0LD,BIRMINGHAM DISABILITY RESOURCE CENTRE,0
4,0.925599,0.436508,0.56361,0,0,0,0,4,0012I00002gRxSB,279699,...,2440645,NORTH KENSINGTON RESIDENTIAL LTD,6066646,45 WARWICK STREET,NaN,WORTHING,WEST SUSSEX,BN11 3DQ,NORTH KENSINGTON RESIDENTIAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,0.6,0,0.515432,0.402778,0,0,0,211,0010G00002JMXIm,NaN,...,18263,1PMOBILE LIMITED,10380951,NETWORK HQ 508 EDGWARE ROAD,THE HYDE,LONDON,ENGLAND,NW9 5AB,1PMOBILE,0
512,0.877115,0,0.587302,0,0,0,0,212,0012I00002R97ze,NaN,...,3297527,SMITHS (GLOUCESTER) LIMITED,4004654,ALKERTON COURT,EASTINGTON,STONEHOUSE,GLOUCESTERSHIRE,GL10 3AQ,SMITHS (GLOUCESTER),0
513,0.612027,0,0.545833,0,0,0,0,213,0012I00002TW5J4,NaN,...,4320257,HP KITCHENS LIMITED,11155068,"1ST FLOOR, CASHS BUSINESS CENTRE",228 WIDDRINGTON ROAD,COVENTRY,WEST MIDLANDS,CV1 4PB,HP KITCHENS,0
514,0.613527,0,0.489881,0,0,0,0,214,0012I00002SfMbQ,NaN,...,1563778,FTD82 LIMITED,SC546429,9 ROYAL CRESCENT,NaN,GLASGOW,NaN,G3 7SP,FTD82,0


Output to CSV

In [21]:
df_combined.to_csv(r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Heggle\Record Linkage - Company Data\Company Data\Outputs\RLW_Sept_Labelled.csv',index=False)